# MultiOutput Random Forest Regressor

In [1]:
import logging
from typing import Any, Callable, Dict, List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
from optuna.trial import FrozenTrial, TrialState
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn import metrics, preprocessing, model_selection, pipeline, ensemble

optuna.logging.set_verbosity(logging.ERROR)

### Define Random State and Test Size

In [2]:
RANDOM_STATE = 107
TEST_SIZE = 0.20

## Read in the Data

In [3]:
data = pd.read_csv("2022_08_29_AllCycles_LHS.csv")
data.head()

,Conductivity,CycleNumber,Porosity,Permeability,AverageFiberDiameter,MeanPoreDiameter,VoltageEfficiency,CoulombicEfficiency,EnergyEfficiency
0,67.326517,2,0.926359,1.700000e-10,0.000014,0.000135,0.746404,0.956847,0.714195
1,67.326517,4,0.926359,1.700000e-10,0.000014,0.000135,0.746284,0.957371,0.714470
2,67.326517,5,0.926359,1.700000e-10,0.000014,0.000135,0.746303,0.957180,0.714346
3,67.326517,6,0.926359,1.700000e-10,0.000014,0.000135,0.746261,0.956961,0.714142
4,86.086664,2,0.818147,3.550000e-11,0.000020,0.000124,0.751707,0.952122,0.715717


### Define Features and Targets

In [4]:
features = [
    "Conductivity",
    "CycleNumber",
    "Porosity",
    "Permeability",
    "AverageFiberDiameter",
    "MeanPoreDiameter"
] 

targets =  [
    "VoltageEfficiency",
    "CoulombicEfficiency",
    "EnergyEfficiency"
]

### Split into Training and Testing

In [5]:
train_data, test_data = model_selection.train_test_split(data, test_size=TEST_SIZE)

In [6]:
train_data.head()

,Conductivity,CycleNumber,Porosity,Permeability,AverageFiberDiameter,MeanPoreDiameter,VoltageEfficiency,CoulombicEfficiency,EnergyEfficiency
380,70.792984,3,0.825773,2.569380e-11,0.000016,0.000197,0.743844,0.950800,0.707247
322,106.910478,2,0.899212,1.380000e-10,0.000019,0.000153,0.758319,0.956976,0.725692
67,105.240878,5,0.943287,5.450000e-10,0.000019,0.000166,0.759658,0.947747,0.719963
275,99.637067,2,0.859639,2.220000e-11,0.000011,0.000133,0.756273,0.950553,0.718878
170,94.810083,5,0.843651,1.720000e-11,0.000011,0.000104,0.755114,0.942012,0.711327


In [7]:
x_train = train_data[features]
y_train = train_data[targets]

x_test = test_data[features]
y_test = test_data[targets]

### Define Model Scoring Metrics 
- Root Mean Squared Error (RMSE)
- Mean Absolute Percentage Error (MAPE)

In [8]:
def print_results(y_true: np.ndarray, y_pred: np.ndarray) -> None:
    print(f"RMSE = {mean_squared_error(y_true, y_pred, squared=False):,.6f}")
    print(f"MAPE = {mean_absolute_percentage_error(y_true, y_pred):.5%}")

## Baseline Model with Multi-Output Random Forest Regressor
**Baseline**: Before Hyperparameter Tuning to get final surrogate model

### Printing Testing Error For Baseline Model (Multi-Output Random Forest Regressor)

In [9]:
model = MultiOutputRegressor(RandomForestRegressor())
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print_results(y_test, y_pred)

RMSE = 0.003868
MAPE = 0.26595%


### Printing Training Error for Baseline Model

In [10]:
y_pred_train = model.predict(x_train)
print_results(y_train, y_pred_train)

RMSE = 0.003604
MAPE = 0.14107%


### Setting Up Optuna Study for Baseline Model

In [11]:
ResponseVector = Tuple[float, float, float, float, float, float]
ObjectiveFunction = Callable[[optuna.Trial], ResponseVector]


def create_objective(model: MultiOutputRegressor) -> ObjectiveFunction:
    def objective(trial: optuna.Trial) -> ResponseVector:
        x = [
            trial.suggest_float("Conductivity", 61.32085384, 109.7652427),
            trial.suggest_float("CycleNumber", 2, 6),
            trial.suggest_float("Porosity", 0.7217427, 0.949060441),
            trial.suggest_float("Permeability", 4.40E-12, 1.14E-09),
            trial.suggest_float("AverageFiberDiameter", 1.00E-05, 2.00E-05),
            trial.suggest_float("MeanPoreDiameter", 0.0001, 0.000198985)
        ]
        x = pd.DataFrame([x], columns=features)
        y = model.predict(x)
        return tuple(y.ravel())

    return objective

In [13]:
study = optuna.study.create_study(
    storage="sqlite:///2022_10_23_MultiOutputRegressor_RFR.db",
    sampler=optuna.samplers.NSGAIISampler(),
    study_name="OptunaTrial_107_RFR",
    load_if_exists=True,
    directions=["maximize" for _ in range(len(targets))],
)

In [14]:
model = MultiOutputRegressor(RandomForestRegressor())
model.fit(x_train, y_train)
objective_function = create_objective(model)

In [15]:
study.optimize(objective_function, n_trials=250, show_progress_bar=True)

/opt/anaconda3/envs/PythonTutorials/lib/python3.7/site-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/250 [00:00<?, ?it/s]

In [16]:
def frozen_trials_to_frame(trials: List[FrozenTrial]) -> pd.DataFrame:
    return pd.DataFrame([frozen_trial_to_dict(trial) for trial in trials])


def frozen_trial_to_dict(trial: FrozenTrial) -> Dict[str, Any]:
    return {
        "number": trial.number,
        "state": trial.state,
        "values": trial.values,
        "datetime_start": trial.datetime_start,
        "datetime_complete": trial.datetime_complete,
        "params": trial.params,
        "distributions": trial.distributions,
        "user_attrs": trial.user_attrs,
        "system_attrs": trial.system_attrs,
        "intermediate_values": trial.intermediate_values,
    }

In [17]:
trials = frozen_trials_to_frame(study.get_trials())
trials = trials.loc[trials["state"] == TrialState.COMPLETE]
trials["max_value"] = trials["values"].apply(np.max)
trials["mean_value"] = trials["values"].apply(np.mean)
trials.sort_values(by=["max_value"], ascending=False, inplace=True)
trials.iloc[0].loc["params"]

{'Conductivity': 73.6244231257345,
 'CycleNumber': 2.5886459106689785,
 'Porosity': 0.918729326383119,
 'Permeability': 3.115785549347731e-10,
 'AverageFiberDiameter': 1.3038991904651788e-05,
 'MeanPoreDiameter': 0.00016026759788531175}

In [18]:
trials.params.iloc[:10]

170    {'Conductivity': 73.6244231257345, 'CycleNumbe...
137    {'Conductivity': 86.72104003106753, 'CycleNumb...
247    {'Conductivity': 72.7363516837437, 'CycleNumbe...
227    {'Conductivity': 106.4103303560874, 'CycleNumb...
239    {'Conductivity': 75.97312322253465, 'CycleNumb...
39     {'Conductivity': 72.7363516837437, 'CycleNumbe...
233    {'Conductivity': 106.4103303560874, 'CycleNumb...
76     {'Conductivity': 86.72104003106753, 'CycleNumb...
183    {'Conductivity': 72.7363516837437, 'CycleNumbe...
199    {'Conductivity': 90.24420233635182, 'CycleNumb...
Name: params, dtype: object

In [19]:
top_ten_design_params = trials.params.iloc[:10]
top_ten_design_params = pd.DataFrame.from_records(top_ten_design_params.to_list(), index=top_ten_design_params.index)
top_ten_design_params

,Conductivity,CycleNumber,Porosity,Permeability,AverageFiberDiameter,MeanPoreDiameter
170,73.624423,2.588646,0.918729,3.115786e-10,0.000013,0.000160
137,86.721040,3.594741,0.799127,8.102601e-10,0.000012,0.000151
247,72.736352,2.798944,0.927858,8.900789e-10,0.000018,0.000160
227,106.410330,3.594741,0.799127,8.102601e-10,0.000012,0.000111
239,75.973123,3.594741,0.925584,1.024343e-09,0.000020,0.000189
39,72.736352,3.803226,0.927858,8.900789e-10,0.000018,0.000132
233,106.410330,2.558359,0.940242,8.466539e-10,0.000013,0.000160
76,86.721040,3.594741,0.799127,7.254447e-10,0.000012,0.000164
183,72.736352,5.947160,0.927858,2.364692e-10,0.000019,0.000151
199,90.244202,2.588646,0.940242,3.115786e-10,0.000013,0.000160


In [20]:
pd.DataFrame(model.predict(top_ten_design_params), columns=targets, index=top_ten_design_params.index)

,VoltageEfficiency,CoulombicEfficiency,EnergyEfficiency
170,0.748313,0.961205,0.717977
137,0.751533,0.961067,0.727792
247,0.748498,0.961041,0.718681
227,0.757507,0.960985,0.731158
239,0.749043,0.960855,0.719346
39,0.749132,0.960146,0.719428
233,0.758560,0.959837,0.726276
76,0.751448,0.959678,0.727155
183,0.748694,0.959669,0.715766
199,0.755793,0.959639,0.720360



# Hyperparameter Tuning Using Bayesian Methods and NSGA-II Optimizer

### Setting up Optuna Study with the Hyperparameters to find the set of Hyperparameters that has the lowest MAPE

In [22]:
def hyperparameter_objective(trial: optuna.Trial) -> float:
    hyperparams = {
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 50),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 200),
        "max_depth": trial.suggest_int("max_depth", 1, 50),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2",None])
    }

    model = MultiOutputRegressor(RandomForestRegressor(
            n_estimators=300,
            random_state=RANDOM_STATE,
            **hyperparams,
        )
    )
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    MAPE = mean_absolute_percentage_error(
        y_test, y_pred, multioutput="uniform_average"#, squared=False
    )
    return MAPE



In [23]:
model_selection_study = optuna.study.create_study(
    storage="sqlite:///model_selection.db",
    sampler=optuna.samplers.NSGAIISampler(),
    study_name="OptunaTrial_107_RFR",
    load_if_exists=True,
    direction="minimize",
)

In [24]:
model_selection_study.optimize(
    hyperparameter_objective, n_trials=200, show_progress_bar=True
)

/opt/anaconda3/envs/PythonTutorials/lib/python3.7/site-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/200 [00:00<?, ?it/s]

### Printing the Optimized Hyperparameter Combination for Multi-Output RFR

Make sure that `ascending=True` because you want to get the hyperparameter combinations that are for the lowest MAPE

In [25]:
trials = frozen_trials_to_frame(model_selection_study.get_trials())
trials = trials.loc[trials["state"] == TrialState.COMPLETE]
trials["values"] = trials["values"].apply(lambda x: x[0])
trials.sort_values(by=["values"], ascending=True, inplace=True)
trials.iloc[0].loc["params"]

{'min_samples_leaf': 2,
 'min_samples_split': 7,
 'max_depth': 33,
 'max_features': None}

### Printing the Training Error and Testing Error for the Surrogate model 
**Surrogate Model**: Machine learning model with optimized set of hyperparameters

In [26]:
best_model = MultiOutputRegressor(
    RandomForestRegressor(
        n_estimators=300,
        random_state=RANDOM_STATE,
        **trials.iloc[0].loc["params"],
    )
)
best_model.fit(x_train, y_train)
y_pred_test = best_model.predict(x_test)
print_results(y_test, y_pred_test)

RMSE = 0.003414
MAPE = 0.26137%


In [27]:
y_pred_train = best_model.predict(x_train)
print_results(y_train, y_pred_train)

RMSE = 0.006764
MAPE = 0.26267%


In [28]:
parameters = trials.iloc[0].loc["params"]
parameters = {f"estimator__{k}": [v] for k,v in parameters.items()}
parameters

{'estimator__min_samples_leaf': [2],
 'estimator__min_samples_split': [7],
 'estimator__max_depth': [33],
 'estimator__max_features': [None]}

In [29]:
from sklearn.model_selection import GridSearchCV

In [30]:
RandomForestRegressor

sklearn.ensemble._forest.RandomForestRegressor

In [31]:
grid_search = GridSearchCV(MultiOutputRegressor(RandomForestRegressor()), parameters)
grid_search

GridSearchCV(estimator=MultiOutputRegressor(estimator=RandomForestRegressor()),
             param_grid={'estimator__max_depth': [33],
                         'estimator__max_features': [None],
                         'estimator__min_samples_leaf': [2],
                         'estimator__min_samples_split': [7]})

In [32]:
grid_search.fit(data[features], data[targets])

GridSearchCV(estimator=MultiOutputRegressor(estimator=RandomForestRegressor()),
             param_grid={'estimator__max_depth': [33],
                         'estimator__max_features': [None],
                         'estimator__min_samples_leaf': [2],
                         'estimator__min_samples_split': [7]})

In [45]:
pd.DataFrame(grid_search.cv_results_).T

,0
mean_fit_time,0.429506
std_fit_time,0.009786
mean_score_time,0.028945
std_score_time,0.001787
param_estimator__max_depth,33
param_estimator__max_features,None
param_estimator__min_samples_leaf,2
param_estimator__min_samples_split,7
params,"{'estimator__max_depth': 33, 'estimator__max_f..."
split0_test_score,0.480926


## Setting Up Study to find Optimal Electrode Parameter Combinations using the Surrogate Model

In [34]:
ResponseVectorOpt = Tuple[float, float, float, float, float, float]
ObjectiveFunctionOpt = Callable[[optuna.Trial], ResponseVectorOpt]


def create_objective(model: MultiOutputRegressor) -> ObjectiveFunctionOpt:
    def objective(trial: optuna.Trial) -> ResponseVectorOpt:
        x = [
            trial.suggest_float("Conductivity", 61.32085384, 109.7652427),
            trial.suggest_float("CycleNumber", 2, 6),
            trial.suggest_float("Porosity", 0.7217427, 0.949060441),
            trial.suggest_float("Permeability", 4.40E-12, 1.14E-09),
            trial.suggest_float("AverageFiberDiameter", 1.00E-05, 2.00E-05),
            trial.suggest_float("MeanPoreDiameter", 0.0001, 0.000198985)
        ]
        x = pd.DataFrame([x], columns=features)
        y = model.predict(x)
        return tuple(y.ravel())

    return objective

In [35]:
study = optuna.study.create_study(
    storage="sqlite:///2022_10_23_MultiOutputRegressor_RFROpt.db",
    sampler=optuna.samplers.NSGAIISampler(),
    study_name="OptunaTrial_107_RFROpt",
    load_if_exists=True,
    directions=["maximize" for _ in range(len(targets))],
)

In [36]:
{'estimator__min_samples_leaf': [2],
 'estimator__min_samples_split': [7],
 'estimator__max_depth': [33],
 'estimator__max_features': [None]}

{'estimator__min_samples_leaf': [2],
 'estimator__min_samples_split': [7],
 'estimator__max_depth': [33],
 'estimator__max_features': [None]}

### Re-Training the Multi-Output Random Forest Regressor with the Optimized Set of Hyperparameters

In [37]:
model_opt = MultiOutputRegressor(RandomForestRegressor(min_samples_leaf=2,
                                                       min_samples_split=7,
                                                       max_depth=33,
                                                       max_features=None))
model_opt.fit(x_train, y_train)
objective_function = create_objective(model)

In [38]:
study.optimize(objective_function, n_trials=200, show_progress_bar=True)

/opt/anaconda3/envs/PythonTutorials/lib/python3.7/site-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/200 [00:00<?, ?it/s]

In [39]:
def frozen_trials_to_frame(trials: List[FrozenTrial]) -> pd.DataFrame:
    return pd.DataFrame([frozen_trial_to_dict(trial) for trial in trials])


def frozen_trial_to_dict(trial: FrozenTrial) -> Dict[str, Any]:
    return {
        "number": trial.number,
        "state": trial.state,
        "values": trial.values,
        "datetime_start": trial.datetime_start,
        "datetime_complete": trial.datetime_complete,
        "params": trial.params,
        "distributions": trial.distributions,
        "user_attrs": trial.user_attrs,
        "system_attrs": trial.system_attrs,
        "intermediate_values": trial.intermediate_values,
    }

Make sure that `ascending=False` to make sure that the combinations that have the maximum efficiencies are first

In [40]:
trials = frozen_trials_to_frame(study.get_trials())
trials = trials.loc[trials["state"] == TrialState.COMPLETE]
trials["max_value"] = trials["values"].apply(np.max)
trials["mean_value"] = trials["values"].apply(np.mean)
trials.sort_values(by=["max_value"], ascending=False, inplace=True)
trials.iloc[0].loc["params"]

{'Conductivity': 68.05050615267574,
 'CycleNumber': 2.6357128241244334,
 'Porosity': 0.8319368227891215,
 'Permeability': 6.273619339271112e-10,
 'AverageFiberDiameter': 1.1049550849906381e-05,
 'MeanPoreDiameter': 0.00011320738640583128}

In [41]:
trials.params.iloc[:10]

126    {'Conductivity': 68.05050615267574, 'CycleNumb...
187    {'Conductivity': 109.44057615512519, 'CycleNum...
30     {'Conductivity': 68.05050615267574, 'CycleNumb...
181    {'Conductivity': 68.05050615267574, 'CycleNumb...
116    {'Conductivity': 68.05050615267574, 'CycleNumb...
153    {'Conductivity': 104.4504757080266, 'CycleNumb...
179    {'Conductivity': 64.61162265472316, 'CycleNumb...
165    {'Conductivity': 104.4504757080266, 'CycleNumb...
99     {'Conductivity': 64.61162265472316, 'CycleNumb...
121    {'Conductivity': 107.03699746026189, 'CycleNum...
Name: params, dtype: object

In [42]:
top_ten_design_params_tuned = trials.params.iloc[:10]
top_ten_design_params_tuned = pd.DataFrame.from_records(top_ten_design_params_tuned.to_list(), index=top_ten_design_params_tuned.index)
top_ten_design_params_tuned

,Conductivity,CycleNumber,Porosity,Permeability,AverageFiberDiameter,MeanPoreDiameter
126,68.050506,2.635713,0.831937,6.273619e-10,0.000011,0.000113
187,109.440576,3.482428,0.831937,3.891610e-10,0.000020,0.000113
30,68.050506,2.635713,0.831937,6.273619e-10,0.000011,0.000143
181,68.050506,2.635713,0.831937,2.442856e-10,0.000011,0.000143
116,68.050506,2.635713,0.831937,3.891610e-10,0.000011,0.000143
153,104.450476,3.269604,0.934283,9.383683e-10,0.000013,0.000160
179,64.611623,2.858764,0.894429,1.055328e-09,0.000019,0.000183
165,104.450476,2.858764,0.831937,7.548453e-10,0.000011,0.000176
99,64.611623,4.297634,0.894429,1.055328e-09,0.000019,0.000189
121,107.036997,2.235964,0.831937,1.134441e-09,0.000020,0.000143


In [43]:
pd.DataFrame(model.predict(top_ten_design_params_tuned), columns=targets, index=top_ten_design_params_tuned.index)

,VoltageEfficiency,CoulombicEfficiency,EnergyEfficiency
126,0.744390,0.959706,0.716197
187,0.758466,0.959614,0.720156
30,0.744365,0.959554,0.716394
181,0.744365,0.959554,0.716394
116,0.744365,0.959554,0.716394
153,0.758636,0.959362,0.725989
179,0.743278,0.959360,0.708996
165,0.756884,0.959073,0.720131
99,0.742972,0.958731,0.707612
121,0.757328,0.958603,0.721956
